# Reconhecimento de Padrões
#### UTFPR — Universidade Tecnológica Federal do Paraná 
#### Nomes: Rafael Menezes Barboza, Carlos Alexandre Peron dos Santos


Este trabalho tem como objetivo buscar por caracteristicas na base "EMG data for gestures Data Set" e assim identificar posições das maos. O link abaixo é referente a base de dados utilizada disponivel para download.
https://archive.ics.uci.edu/ml/datasets/EMG+data+for+gestures#

O codigo fonte esta disponivel publicamente no GitHub e pode ser acessado pelo link abaixo.
https://github.com/rmmenezes/ReconhecimentoDePadroes-UTFPR.git

### Import das bibliotecas e estruturas necessárias

In [1]:
import os
import numpy as np
import math
from librosa import stft
from scipy.signal import welch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

X = []
y = []
caracteristicas = []
classes = []

### Leitura da base de dados.

Os arquivos estão estruturados em 36 diretórios, equivalente aos 36 participantes do experimento.
Cada diretório possui dois arquivos, equivalentes aos trials de cada participante.
Cada arquivo possui leituras de 8 canais conectados ao ante-braço dos participantes durante um período de tempo não fixo. As leituras foram realizadas movimento à movimento alternados com intervalos de "descanço" (movimento não classificado - repouso).

Por exemplo: leituras da classe 1, leituras durante repouso, leitruas da class 2, leituras durante repouso, leitruas da class 3...

In [2]:
diretorio_db = "../data/EMG_data_for_gestures-master"
pastas = os.listdir(diretorio_db)
for pasta in pastas:
    arquivos = os.listdir(diretorio_db + '/' + pasta)
    for arquivo in arquivos:
        #le o arquivo e divide linha por linha
        conteudo = open(diretorio_db + '/' + pasta + '/' + arquivo).read().split('\n')
        #remove ultima linha (linha em branco)
        conteudo.pop()
        #remove primeira linha (cabecalho)
        conteudo.pop(0)
        classe_atual = '0'
        experimento_caracteristicas = []
        experimento_classes = []
        for i in range(len(conteudo)):
            conteudo[i] = conteudo[i].split('\t')
            #remove primeira linha (campo "time")
            conteudo[i].pop(0)
            if conteudo[i][-1] != classe_atual:
                caracteristicas.append(experimento_caracteristicas)
                if(len(experimento_classes)) == 0:
                    classes.append('0')
                else:
                    classes.append(experimento_classes[0])
                classe_atual = conteudo[i][-1]

                experimento_caracteristicas = []
                experimento_classes = []
                experimento_caracteristicas.append(conteudo[i][:-1])
                experimento_classes.append(conteudo[i][-1])
            else:
                experimento_caracteristicas.append(conteudo[i][:-1])
                experimento_classes.append(conteudo[i][-1])
            
        caracteristicas.append(experimento_caracteristicas)
        classes.append(experimento_classes[0])
        

for i in range(len(caracteristicas)):
    for j in range(len(caracteristicas[i])):
        for k in range(len(caracteristicas[i][j])):
            caracteristicas[i][j][k] = float(caracteristicas[i][j][k])
        caracteristicas[i][j] = np.array(caracteristicas[i][j])
    caracteristicas[i] = np.array(caracteristicas[i])
    classes[i] = np.array(classes[i])
caracteristicas = np.array(caracteristicas)
classes = np.array(classes)

print(caracteristicas.shape, classes.shape)

(1816,) (1816,)


### Extração de informação primária necessária para cálculo das características no domínio do tempo.

In [3]:
quantos_valores = []

somas_geral = []
somas_temp = []

somas_quadraticas = []
somas_quadraticas_temp = []

somas_quadraticas_sem_modulo = []
somas_quadraticas_sem_modulo_temp = []

somas_wf = []
somas_wf_temp = []

for i in range(8):
    somas_temp.append(0.0)
    somas_quadraticas_temp.append(0.0)
    somas_wf_temp.append(0.0)
    somas_quadraticas_sem_modulo_temp.append(0.0)

for experimento in caracteristicas:
    for tipo in experimento:
        cont = 0
        for i in range(len(tipo)):
            somas_temp[i] += abs(float(tipo[i]))
            somas_quadraticas_temp[i] += abs(float(tipo[i])) ** 2
            somas_quadraticas_sem_modulo_temp[i] += float(tipo[i]) ** 2
            cont += 1
            if(cont < len(tipo) - 1):
                somas_wf_temp[i] = abs(float(tipo[i+1]) - float(tipo[i]))
    somas_geral.append(somas_temp)
    somas_quadraticas.append(somas_quadraticas_temp)
    somas_wf.append(somas_wf_temp)
    somas_quadraticas_sem_modulo.append(somas_quadraticas_sem_modulo_temp)
    
    quantos_valores.append(cont)

### Extração das características no domínio do tempo.

In [4]:
iemg = np.array(somas_geral)

mav = np.array(somas_geral)
for i in range(len(mav)):
    for j in range(len(mav[i])):
        mav[i][j] *= 1/len(quantos_valores)

ssi = np.array(somas_quadraticas)

mavs = mav
for i in range(len(mavs)-1):
    mavs[i] = mavs[i+1] - mavs[i]

var = np.array(somas_quadraticas_sem_modulo)
for i in range(len(var)):
    for j in range(len(var[i])):
        var[i][j] *= 1/(len(quantos_valores) - 1)

rms = ssi
for i in range(len(rms)):
    for j in range(len(rms[i])):
        rms[i][j] *= math.sqrt(1/len(quantos_valores))
        
wf = np.array(somas_wf)

### Transformação do sinal para o domínio da frequência, e mais extração de caracteristicas.

In [47]:
from scipy.signal import stft
# print(caracteristicas.shape)
# print(caracteristicas[0].shape)
# print(caracteristicas[1].shape)
# print(caracteristicas[2].shape)
# print(caracteristicas[3].shape)


for i in range(1816):
    caracteristicas[i] = np.swapaxes(caracteristicas[i], 0, 1)
#     print(caracteristicas[i].shape)
    _, _, w = stft(caracteristicas[i], fs=4000, nperseg=512, noverlap=256)
    print(w.shape)


(8, 257, 8)


ValueError: noverlap must be less than nperseg.

In [5]:
fmd = []
fmn = []
cont = 0
for experimento in caracteristicas:
    cont += 1
    if len(experimento) == 0:
        fmd.append(np.zeros(8))
        fmn.append(np.zeros(8))
    else:
        somatorio_fmd = 0
        somatorio_fmn = 0
        fmd_temp = []
        fmn_temp = []
        for i in range(len(experimento[0])):
            passo_necessario = np.asfortranarray(experimento[:,i])
            transformada = np.abs(stft(passo_necessario, n_fft=512, hop_length=512))
            psd = welch(transformada)
            Fi = (cont * 5000) / (2 * len(psd[1]))
            for linha in psd[1]:
                somatorio_fmd += linha.sum()
                somatorio_fmn += linha.sum()
            fmn_temp.append((Fi * somatorio_fmn) / somatorio_fmn)
            fmd_temp.append(0.5 * somatorio_fmd)
        fmd_temp = np.array(fmd_temp)
        fmd.append(fmd_temp)
        fmn_temp = np.array(fmn_temp)
        fmn.append(fmn_temp)
fmd = np.array(fmd)
fmn = np.array(fmn)

/home/todos/alunos/cm/a834173/Documentos/ReconhecimentoDePadroes-UTFPR/lib/python3.5/site-packages/scipy/signal/spectral.py:1969: UserWarning: nperseg = 256 is greater than input length  = 4, using nperseg = 4
  .format(nperseg, input_length))
/home/todos/alunos/cm/a834173/Documentos/ReconhecimentoDePadroes-UTFPR/lib/python3.5/site-packages/scipy/signal/spectral.py:1969: UserWarning: nperseg = 256 is greater than input length  = 3, using nperseg = 3
  .format(nperseg, input_length))
/home/todos/alunos/cm/a834173/Documentos/ReconhecimentoDePadroes-UTFPR/lib/python3.5/site-packages/scipy/signal/spectral.py:1969: UserWarning: nperseg = 256 is greater than input length  = 6, using nperseg = 6
  .format(nperseg, input_length))
/home/todos/alunos/cm/a834173/Documentos/ReconhecimentoDePadroes-UTFPR/lib/python3.5/site-packages/scipy/signal/spectral.py:1969: UserWarning: nperseg = 256 is greater than input length  = 5, using nperseg = 5
  .format(nperseg, input_length))
/home/todos/alunos/cm/a8

### Normalização das caracteristicas e divisão da base para treino (contendo 80% das amostras) e teste (contendo os 20% restantes).

In [6]:
# iemg = iemg.mean(axis=1)
# mav = mav.mean(axis=1)
# ssi = ssi.mean(axis=1)
# mavs = mavs.mean(axis=1)
# var = var.mean(axis=1)
# rms = rms.mean(axis=1)
# wf = wf.mean(axis=1)

# fmd = fmd.mean(axis=1)
# fmn = fmn.mean(axis=1)


X = np.concatenate((iemg, mav, ssi, mavs, var, rms, wf''', fmd, fmn'''), axis=1)
y = classes

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=True)

ss = StandardScaler()
ss.fit(X_train)
ss.transform(X_train)
ss.transform(X_test)

array([[ 1.        ,  1.        ,  1.        , ..., -1.02593995,
        -1.02593995, -1.02593995],
       [ 1.        ,  1.        ,  1.        , ...,  1.11155719,
         1.11155719,  1.11155719],
       [ 1.        ,  1.        ,  1.        , ...,  0.31307778,
         0.31307778,  0.31307778],
       ...,
       [ 1.        ,  1.        ,  1.        , ...,  0.2998014 ,
         0.2998014 ,  0.2998014 ],
       [ 1.        ,  1.        ,  1.        , ...,  0.50084372,
         0.50084372,  0.50084372],
       [ 1.        ,  1.        ,  1.        , ..., -0.19711453,
        -0.19711453, -0.19711453]])

Classificação dos sinais.
0 - dados não marcados,
1 - mão em repouso,
2 - mão fechada em punho,
3 - flexão do punho,
4 - extensão do punho,
5 - desvios radiais,
6 - desvios ulnares,
7 - palma estendida (o gesto não foi realizado por todos os sujeitos)

### Classificação.

Classificação dos sinais:
0 - dados não marcados;
1 - mão em repouso;
2 - mão fechada em punho;
3 - flexão do punho;
4 - extensão do punho;
5 - desvios radiais;
6 - desvios ulnares;
7 - palma estendida (o gesto não foi realizado por todos os sujeitos).

Utilização dos classificadores SVM e LDA.
Para o classificador SVM, foram testados os valores de gamma 0.001, 0.01, 0.1, os valores de C 1, 10, 100, 1000, e os kernels "rbf" e "linear".

Para o classificador LDA foram testados os solvers "svd" e "lsqr", mas descartado o "svd" por apresentar resultado muito abaixo do esperado.

In [7]:
for kernel in ['rbf']:#, 'linear']:
    for gamma in [0.1, 0.01, 0.001]:
        for C in [1, 10, 100, 1000]:
            classificador = None
            classificador = svm.SVC(gamma=gamma, C=C, kernel=kernel).fit(X_train, y_train)
            print('acuracia:', (classificador.score(X_test, y_test)) * 100, 'kernel:', kernel, 'gamma:', gamma, 'C:', C)
            

cls = None
cls = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto', n_components=7).fit(X_train, y_train)
print('\nLDA acuracia:', cls.score(X_test, y_test) * 100)

acuracia: 53.2967032967033 kernel: rbf gamma: 0.1 C: 1
acuracia: 53.2967032967033 kernel: rbf gamma: 0.1 C: 10
acuracia: 53.2967032967033 kernel: rbf gamma: 0.1 C: 100
acuracia: 53.2967032967033 kernel: rbf gamma: 0.1 C: 1000
acuracia: 53.2967032967033 kernel: rbf gamma: 0.01 C: 1
acuracia: 53.2967032967033 kernel: rbf gamma: 0.01 C: 10
acuracia: 53.2967032967033 kernel: rbf gamma: 0.01 C: 100
acuracia: 53.2967032967033 kernel: rbf gamma: 0.01 C: 1000
acuracia: 53.2967032967033 kernel: rbf gamma: 0.001 C: 1
acuracia: 4.670329670329671 kernel: rbf gamma: 0.001 C: 10
acuracia: 4.670329670329671 kernel: rbf gamma: 0.001 C: 100
acuracia: 4.670329670329671 kernel: rbf gamma: 0.001 C: 1000

LDA acuracia: 53.02197802197802


In [10]:
# from sklearn.feature_selection import RFE
# from sklearn.svm import SVR
# classif = SVR(kernel='rbf')
# selector = RFE(classif, 36, step = 2)
# selector = selector.fit_transform(X_train, y_train)
# print(selector.score(X_test, y_test))

In [11]:
# from sklearn.svm import LinearSVR
# cl = LinearSVR().fit(X_train, y_train)
# print(cl.score(X_test, y_test) * 1000)